In [ ]:
import torch
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# load data and predictions
y_hat = # predictions
y = # true locations

n, m = [*y.shape]

In [ ]:
# interocular distance d
d = 

In [ ]:
""" Accuracy Metrics: Normalized Mean Error (NME) and CED/AUC
y, y_hat have dimensions of num_samples x (2*num_features)
d is the interocular distance
"""
# outputs the NME of each image in the dataset
def nme(y_hat, y, m, d):
    return torch.square(torch.norm(y_hat-y, dim=1))/ d / m

# outputs the average NME across all images
def total_nme(nme):
    return torch.mean(NME)

def cdf(nme):
    x = torch.sort(NME)
    y = np.arange(n) / float(n)
    plt.xlabel('x-axis')
    plt.ylabel('y-axis')
  
    plt.title('Plot of NME vs Fraction of Samples')
    plt.plot(x, y, marker='o')
    plt.show

def auc(nme, alpha):
    x = torch.where(NME < alpha, 1, 0)
    return torch.sum(failure) / n
    
def fr(nme, alpha):
    return 1-auc(nme, alpha)

In [ ]:
nme = nme(y_hat, y, m, d)
total_nme = total_nme(nme)

In [ ]:
cdf = cdf(nme)

In [ ]:
"""Bias Metrics: Majority/Minority Difference (MMD) 
g = group (as column number)
nme = num_samples dimension tensor
"""
def mmd(g, nme, data):
    acc = 1-nme
    majority = acc[tensor.where(data[:, g] = 1)]
    minority = acc[tensor.where(data[:, g] = 0)]
    difference = torch.mean(majority) - torch.mean(minority)
    return difference

# def iostm(g, nme, )